In [1]:
import os
import numpy as np
import pandas as pd
import zipfile
import json
import matplotlib.pyplot as plt
pd.options.display.max_rows = 330

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [2]:
data = pd.read_pickle('Preprocessed_2021-03-15.pkl')

pilot_participants = ['gv_m_26','js_m_27','mb_f_30','tt_m_23', 'ys_m_29.1']
experiment_participants = ['al_m_23','bd_f_25','el_m_26','gh_m_29','lb_f_19',\
                               'mb_f_28','om_m_31','rl_f_24',\
                               'rt_f_21','sf_m_24','ts_f_26',]

participants = pilot_participants[0]
participants+=experiment_participants

eeg_cols = []
eeg_cols += ["baf_"+str(x) for x in [*range(1,122)]]
eeg_cols += ["tf_"+str(x) for x in [*range(1,104)]]
eeg_cols += ["VC_"+str(x) for x in [*range(0,13)]]
eeg_cols += ["A_"+str(x) for x in [*range(0,14)]]
    
# Hard code section to select participants 

df = data[data['participant']==participants[5]]
window=5
# df = data   

# The EEG data is a matrix of 1673 rows and 251 columns. Need to reshape to 144 x WINDOW x 251
# then we can crop it down
loaded = list()
y = list()

for a in df.uniqueTrialCounter.unique(): #runs from 0 to 143
    #load x element
    b = np.array(data.loc[data.uniqueTrialCounter == a,'baf_1':'A_13'])[:window,:]
    loaded.append(b)
    #load y element
    myindex = data[data['uniqueTrialCounter']==a].index[0]
    y.append(data.iloc[myindex]['label'])

loaded = keras.preprocessing.sequence.pad_sequences(loaded, dtype="float",)
loaded = np.stack(loaded)
print(loaded.shape)
X = loaded

y = np.array(y)

print(y.shape)
y = y.reshape(len(y),1)
y.shape
#     y = 
    
print(2*y.shape[0]//3)
    
# np.max(X)
normalizer = Normalization(axis=-1)
normalizer.adapt(X)
    
Xn = normalizer(X)
    
trainX = Xn[0:2*y.shape[0]//3]/np.max(Xn)
testX = Xn[1+2*y.shape[0]//3:]/np.max(Xn)
trainy = y[0:2*y.shape[0]//3]
testy = y[1+2*y.shape[0]//3:]

#     trainy=to_categorical(trainy)
    #     testy=to_categorical(testy)
    # zero-offset class values
#     trainy = trainy - 1
#     testy = testy - 1
    # one hot encode y
trainy = to_categorical(trainy)
testy = to_categorical(testy)
    

print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    


TypeError: can only concatenate str (not "list") to str

In [ ]:
def generateModel1(n_timesteps, n_features,n_outputs):
    model = Sequential()
    model.add(LSTM(100,  input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

def generateModel2(n_timesteps, n_features,n_outputs):
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps ,n_features)))
# #     model.add(LSTM(100))
    model.add(Dropout(0.2))
#     model.add(LSTM(100))
#     model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [28]:

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


In [40]:
repeats=10
# repeat experiment
scores = list()
for r in range(repeats):
    # load data
    trainX, trainy, testX, testy = get_data(0, 'within', 9)
    
    verbose, epochs, batch_size = 0, 50, 200
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    print(n_timesteps, n_features, n_outputs)
    model = generateModel1(n_timesteps ,n_features, n_outputs)
 
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, score = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
    # summarize results
summarize_results(scores)



(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (47, 9, 251) (96, 3)
9 251 3
>#1: 44.681
(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (47, 9, 251) (96, 3)
9 251 3
>#2: 40.426
(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (47, 9, 251) (96, 3)
9 251 3
>#3: 42.553
(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (47, 9, 251) (96, 3)
9 251 3
>#4: 36.170
(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (47, 9, 251) (96, 3)
9 251 3
>#5: 38.298
(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (47, 9, 251) (96, 3)
9 251 3
>#6: 40.426
(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (47, 9, 251) (96, 3)
9 251 3
>#7: 40.426
(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (47, 9, 251) (96, 3)
9 251 3
>#8: 40.426
(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (47, 9, 251) (96, 3)
9 251 3
>#9: 40.426
(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (47, 9, 251) (96, 3)
9 251 3
>#10: 34.043
[44.68085169792175, 40.425533056259155, 42.553192377090454, 36.17021143436432, 38.297873735427856, 40.425533056259155, 40.425533056259155, 

In [35]:
testX[0]
# model.predict((testX[0]))

<tf.Tensor: shape=(6, 251), dtype=float32, numpy=
array([[ 0.40519565,  0.26390797,  0.2655608 , ...,  0.119133  ,
         0.13291448,  0.41024023],
       [ 0.39556843,  0.26244253,  0.20430121, ..., -0.07544658,
         0.1272642 ,  0.3301033 ],
       [ 0.3535288 , -0.1323592 , -0.18463203, ...,  0.06584046,
        -0.00892712, -0.12109778],
       [ 0.21057819, -0.1419756 , -0.04721254, ...,  0.03034545,
        -0.02410885,  0.0777398 ],
       [-0.02604828, -0.18671046,  0.01773684, ..., -0.13395451,
        -0.08709727,  0.07572434],
       [ 0.06540973, -0.0554717 ,  0.1254056 , ..., -0.07783575,
        -0.10648037,  0.11190312]], dtype=float32)>

In [25]:
trainy

array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1